<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/tool/Hybrid_Demucs_Music_Source_Separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPLIT MUSIC INTO STEMS (VOICE, BASS, etc)

## Hybrid Demucs from Colab, requires GPU

[source link](https://colab.research.google.com/drive/1dC9nVxk3V_VPjUADsnFu8EiT-xnU1tGH)

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)
This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-k002q_w1/demucs_db3d6c24b2a34c47bd00509cfc8f83fc
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-k002q_w1/demucs_db3d6c24b2a34c47bd00509cfc8f83fc
  Resolved https://github.com/facebookresearch/demucs to commit e976d93ecc3865e5757426930257e200846a520a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 903.4 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 6.0 MB/s 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Customize the following options!
model = "htdemucs_ft"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/content/drive/MyDrive/demucs_start/'
out_path = '/content/drive/MyDrive/demucs_done/'

In [ ]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [ ]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

In [ ]:
# This is manual upload and download :)
from_upload()
!zip -r separated.zip separated
files.download('./separated.zip')

## Alternative 2, doesnt require GPU

[source link](https://colab.research.google.com/drive/1nLVmRk3Je_v965fsLmYtRGpLqW3Nt37M)


In [ ]:
# @title Set up

# @markdown **TLDR**; upload your audio files a folder named `inputs`, and hit `Ctrl + F9`.
# @markdown
# @markdown Once done (a matter of minutes), separated music sources will become available under `outputs`.

# @markdown **`inputs_folder`**: Choose which folder to take unseparated music from. Supported files include `.mp3`, `.wav`, `.ogg` or `.flac`.
inputs_folder = 'inputs' # @param {type:"string"}

# @markdown **`outputs_folder`**: Choose which folder separated sources will be output to.
outputs_folder = 'outputs' # @param {type:"string"}

# @markdown **`karaoke_mode`**: Separate vocals from the rest of the accompaniment. This will mix the files after separating the mix fully, so this won't be faster or use less memory.
karaoke_mode = False # @param {type:"boolean"}

# @markdown **`output_format`**: How outputs will be saved.
output_format = "mp3 - 160 kbps" # @param ["mp3 - 160 kbps", "mp3 - 320 kbps", "wav - 16 bit", "wav - 24 bit", "wav - 32 bit (float)"] {allow-input: true}

# @markdown **`model`**: Choose which model is used for separation:
# @markdown - `htdemucs`: first version of Hybrid Transformer Demucs. Trained on MusDB + 800 songs. Default model.
# @markdown - `htdemucs_ft`: fine-tuned version of `htdemucs`, separation will take 4 times more time
# @markdown     but might be a bit better. Same training set as `htdemucs`.
# @markdown - `htdemucs_6s`: 6 sources version of `htdemucs`, with `piano` and `guitar` being added as sources.
# @markdown     Note that the `piano` source is not working great at the moment.
# @markdown - `hdemucs_mmi`: Hybrid Demucs v3, retrained on MusDB + 800 songs.
# @markdown - `mdx`: trained only on MusDB HQ, winning model on track A at the [MDX][mdx] challenge.
# @markdown - `mdx_extra`: trained with extra training data (**including MusDB test set**), ranked 2nd on the track B
# @markdown     of the [MDX][mdx] challenge.
# @markdown - `mdx_q`, `mdx_extra_q`: quantized version of the previous models. Smaller download and storage
# @markdown     but quality can be slightly worse.
# @markdown - `SIG`: where `SIG` is a single model from the [model zoo](docs/training.md#model-zoo).
# @markdown
# @markdown [mdx]: https://www.aicrowd.com/challenges/music-demixing-challenge-ismir-2021

model = 'htdemucs_ft' # @param ["htdemucs", "htdemucs_ft", "htdemucs_6s", "hdemucs_mmi", "mdx", "mdx_extra", "mdx_q", "mdx_extra_q"] {allow-input: true}

!mkdir -p {inputs_folder}

### Separate music sources

In [ ]:
#@title Install dependenies

has_gpu = !nvidia-smi > /dev/null 2>&1 && echo 1
if has_gpu:
  print("Using GPU packages")
  !pip install -qU git+https://github.com/adefossez/demucs#egg=demucs
else:
  # avoid installing dependencies to get rid of CUDA dependency
  print("Using CPU packages; separation will be slow. Consider switching to a GPU-enabled runtime")
  !git clone https://github.com/adefossez/demucs.git
  !pip install -q -e demucs -r demucs/requirements_minimal.txt --no-deps
  !pip install -q treetable omegaconf submitit retrying
  !pip install -q torch==2.0.1 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cpu

# Test that everything is in place
!python3 -m demucs.separate --list-models

In [ ]:

# @title Run model

import re

inputs = !find {inputs_folder} -name *.mp3 -o -name *.flac -o -name *.wav -o -name *.ogg || echo error
if inputs[-1] == "error":
  raise RuntimeError(f'Please make sure you have uploaded your audio files under "{inputs_folder}" folder')

extra_flags = ""
if karaoke_mode:
  extra_flags += " --two-stems=vocals"
if output_format == "wav - 16 bit":
  pass
elif output_format == "wav - 24 bit":
  extra_flags += " --int24"
elif output_format == "wav - 32 bit (float)":
  extra_flags += " --float32"
elif output_format.startswith("mp3"):
  bitrate = int(re.search('([0-9]{2,3}) kbps', output_format).group(1))
  extra_flags += f" --mp3 --mp3-bitrate={bitrate}"

inputs_str = " ".join(f'"{s}"' for s in inputs)

!python3 -m demucs.separate -o {outputs_folder} -n {model} {extra_flags} {inputs_str}